In [1]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader, DistributedSampler

import sys
sys.path.append('..')
import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model

/home/shhan/.local/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (5.1.0)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [2]:
parser = argparse.ArgumentParser('Set transformer detector', add_help=False)

# Model parameters
parser.add_argument('--frozen_weights', type=str, default=None,
                        help="Path to the pretrained model. If set, only the mask head will be trained")
# * Backbone
parser.add_argument('--backbone', default='resnet50', type=str,
                        help="Name of the convolutional backbone to use")
parser.add_argument('--dilation', action='store_true',
                        help="If true, we replace stride with dilation in the last convolutional block (DC5)")
parser.add_argument('--position_embedding', default='sine', type=str, choices=('sine', 'learned'),
                        help="Type of positional embedding to use on top of the image features")
# * Transformer
parser.add_argument('--enc_layers', default=6, type=int,
                        help="Number of encoding layers in the transformer")
parser.add_argument('--dec_layers', default=6, type=int,
                        help="Number of decoding layers in the transformer")
parser.add_argument('--dim_feedforward', default=2048, type=int,
                        help="Intermediate size of the feedforward layers in the transformer blocks")
parser.add_argument('--hidden_dim', default=256, type=int,
                        help="Size of the embeddings (dimension of the transformer)")
parser.add_argument('--dropout', default=0.1, type=float,
                        help="Dropout applied in the transformer")
parser.add_argument('--nheads', default=8, type=int,
                        help="Number of attention heads inside the transformer's attentions")
parser.add_argument('--num_queries', default=100, type=int,
                        help="Number of query slots")
parser.add_argument('--pre_norm', action='store_true')

# * Segmentation
parser.add_argument('--masks', action='store_true',
                        help="Train segmentation head if the flag is provided")

# Loss
parser.add_argument('--no_aux_loss', dest='aux_loss', action='store_false',
                        help="Disables auxiliary decoding losses (loss at each layer)")
# * Matcher
parser.add_argument('--set_cost_class', default=1, type=float,
                        help="Class coefficient in the matching cost")
parser.add_argument('--set_cost_bbox', default=5, type=float,
                        help="L1 box coefficient in the matching cost")
parser.add_argument('--set_cost_giou', default=2, type=float,
                        help="giou box coefficient in the matching cost")
# * Loss coefficients
parser.add_argument('--mask_loss_coef', default=1, type=float)
parser.add_argument('--dice_loss_coef', default=1, type=float)
parser.add_argument('--bbox_loss_coef', default=5, type=float)
parser.add_argument('--giou_loss_coef', default=2, type=float)
parser.add_argument('--eos_coef', default=0.1, type=float,
                        help="Relative classification weight of the no-object class")

# dataset parameters
parser.add_argument('--dataset_file', default='coco') # dataset
parser.add_argument('--coco_path', type=str)
parser.add_argument('--coco_panoptic_path', type=str)
parser.add_argument('--remove_difficult', action='store_true')

parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')

# wsod
parser.add_argument('--wsod', action='store_true', help='enable weakly-supervised object detection (WSOD))')
parser.add_argument('--arch', default='vit_small', type=str,
                        choices=['vit_tiny', 'vit_small', 'vit_base'], help='Architecture (support only ViT atm).')
parser.add_argument('--patch_size', default=8, type=int, help='Patch resolution of the model.')
parser.add_argument("--checkpoint_key", default="teacher", type=str,
                        help='Key to use in the checkpoint (example: "teacher")')
parser.add_argument("--pkl", default="../data/coco_train2017_proposals.pkl", type=str, help='directory for proposal')
parser.add_argument("--cache", action='store_true', help="cacheing the labels")

args = parser.parse_args(args=[])
utils.init_distributed_mode(args)
args.dataset_file = "coco"
args.wsod = False #####
args.coco_path = "../data/coco/"
args.cache = True
args

Not using distributed mode


Namespace(arch='vit_small', aux_loss=True, backbone='resnet50', bbox_loss_coef=5, cache=True, checkpoint_key='teacher', coco_panoptic_path=None, coco_path='../data/coco/', dataset_file='coco', dec_layers=6, dice_loss_coef=1, dilation=False, dim_feedforward=2048, distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, mask_loss_coef=1, masks=False, nheads=8, num_queries=100, output_dir='', patch_size=8, pkl='../data/coco_train2017_proposals.pkl', position_embedding='sine', pre_norm=False, remove_difficult=False, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, wsod=False)

In [2]:
import pickle
# load
with open('../data/coco_train2017_proposals.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('../data/coco_val2017_proposals.pkl', 'rb') as f:
    val_data = pickle.load(f)

print(len(train_data))
print(len(val_data))

118281
4991


In [3]:
train_data[391895]

array([[12., 57., 32., 22.],
       [20.,  0., 20., 44.],
       [20., 73.,  2.,  2.],
       [20., 77.,  2.,  2.],
       [22., 75.,  3.,  0.],
       [23., 20.,  0.,  2.],
       [24., 21.,  2.,  6.],
       [24., 69.,  2.,  0.],
       [25., 17.,  1.,  2.],
       [29.,  2.,  5.,  7.],
       [ 0., 36., 15., 43.],
       [30., 47., 14.,  7.],
       [33., 56.,  1.,  2.],
       [19., 34.,  6.,  9.],
       [20., 10.,  0.,  5.],
       [21., 50.,  1.,  3.],
       [22., 22.,  6.,  1.],
       [23., 75.,  2.,  1.],
       [24., 69.,  2.,  2.],
       [25., 63.,  1.,  2.],
       [29.,  0., 15., 49.],
       [31.,  2.,  3.,  7.],
       [32., 61.,  0.,  2.],
       [34., 53., 10., 20.],
       [12., 57., 32., 22.],
       [20.,  0., 13., 44.],
       [20., 79.,  2.,  0.],
       [25., 36.,  1.,  2.],
       [31.,  0.,  9.,  9.],
       [36., 49.,  3.,  1.],
       [ 2., 43., 38., 18.],
       [16.,  0.,  4., 25.],
       [19., 27.,  1.,  2.],
       [20., 77.,  2.,  1.],
       [22., 5

In [5]:
sorted_train_data = sorted(train_data.items())
print(sorted_train_data[0][0])
print(sorted_train_data[-1][0])

9
581929


In [6]:
sorted_train_data[:2] # tuples in a list

[(9,
  array([[34., 56., 25., 23.],
         [ 0., 44.,  2.,  2.],
         [ 0., 44., 19., 27.],
         [ 8., 77.,  6.,  1.],
         [17., 75.,  2.,  0.],
         [20., 53.,  4.,  2.],
         [20., 63., 32., 16.],
         [29.,  8., 30., 62.],
         [45.,  0.,  4.,  2.],
         [53.,  0.,  6.,  3.],
         [55., 76.,  1.,  2.],
         [ 0., 68.,  1.,  2.],
         [ 0., 75.,  3.,  4.],
         [ 4., 32.,  1.,  2.],
         [ 7., 76.,  8.,  3.],
         [ 9., 21.,  2.,  1.],
         [ 9., 44.,  2.,  0.],
         [10., 13., 29., 32.],
         [13., 47.,  4.,  1.],
         [14., 55.,  6., 13.],
         [16., 74.,  9.,  5.],
         [18., 45.,  4.,  4.],
         [18., 64.,  4.,  6.],
         [21., 21.,  3.,  5.],
         [24., 48.,  0.,  2.],
         [26., 65.,  1.,  3.],
         [28., 62.,  3.,  2.],
         [30., 64.,  2.,  4.],
         [31., 30.,  1.,  3.],
         [34.,  0.,  7.,  7.],
         [34., 64.,  5.,  1.],
         [37.,  8.,  1.,  3.],
   

In [ ]:
import logging
proposals = sorted(train_data.items())

ref_proposals = []
for i, boxes in enumerate(proposals['boxes']):
	if i % 2500 == 0:
		logger.info(' {:d}/{:d}'.format(i + 1, len(proposals['boxes'])))
	
	annotation = self.annotations[i]
	assert annotation['img_key'] == proposals['indexes'][i]
	# Remove duplicate boxes and very small boxes and then take top k
	boxes = box_utils.clip_boxes_to_image(boxes, annotation['height'], annotation['width'])
	keep = box_utils.unique_boxes(boxes)
	boxes = boxes[keep, :]
	keep = box_utils.filter_small_boxes(boxes, cfg.FAST_RCNN.MIN_PROPOSAL_SIZE)
	boxes = boxes[keep, :]
	if cfg.FAST_RCNN.TOP_K > 0:
		boxes = boxes[:cfg.FAST_RCNN.TOP_K, :]
	ref_proposals.append(boxes.astype(np.float))

In [7]:
dataset_train = build_dataset(image_set='train', args=args) # wsod setting
dataset_val = build_dataset(image_set='val', args=args) # detection setting for evaluation

../data/coco/images/train2017
loading annotations into memory...
Done (t=16.95s)
creating index...
index created!
hello


In [5]:
dataset_train

In [4]:
ann_file = '/home/shhan/oicr/data/coco/annotations/instances_val2017.json' # annotation path
img_folder = "/home/shhan/oicr/data/coco/images/val2017" # data path

In [5]:
from datasets.coco import CocoDetection, make_coco_transforms

image_set = 'train'
dataset = CocoDetection(img_folder, ann_file, transforms=make_coco_transforms(image_set), return_masks=False)

loading annotations into memory...
Done (t=0.55s)
creating index...
index created!
hello


In [6]:
dataset[0]

(tensor([[[ 0.6906,  0.7248,  0.7591,  ...,  0.8447,  0.8447,  0.8276],
          [ 0.7077,  0.7248,  0.7419,  ...,  0.8276,  0.8276,  0.8276],
          [ 0.7077,  0.7248,  0.7419,  ...,  0.8276,  0.8276,  0.8276],
          ...,
          [ 1.1015,  1.1015,  1.1015,  ...,  0.3481,  0.3652,  0.3994],
          [ 1.1015,  1.1187,  1.1358,  ...,  0.4166,  0.4337,  0.4508],
          [ 1.1187,  1.1358,  1.1529,  ...,  0.4679,  0.4679,  0.4851]],
 
         [[ 0.2402,  0.2752,  0.3102,  ...,  0.7129,  0.7129,  0.6954],
          [ 0.2577,  0.2752,  0.2927,  ...,  0.6954,  0.6954,  0.6954],
          [ 0.2577,  0.2752,  0.2927,  ...,  0.6954,  0.6954,  0.6954],
          ...,
          [ 0.7829,  0.7654,  0.7479,  ..., -0.5476, -0.5126, -0.4601],
          [ 0.8880,  0.8704,  0.8704,  ..., -0.4776, -0.4601, -0.4076],
          [ 0.9580,  0.9405,  0.9405,  ..., -0.4426, -0.4251, -0.3901]],
 
         [[-0.6018, -0.5670, -0.5321,  ..., -0.0964, -0.0964, -0.1138],
          [-0.5844, -0.5670,

In [68]:
dataset[0][1].append('img_labels': torch.unique(dataset[0][1]['labels']))
print(torch.unique(dataset[0][1]['labels']))

SyntaxError: invalid syntax (1945514742.py, line 1)

In [70]:
d = dataset[0][1]
print(d)
d['img_labels'] = torch.unique(d[0][1]['labels'])
d

{'boxes': tensor([[0.6104, 0.4161, 0.0386, 0.1631],
        [0.8724, 0.5052, 0.2333, 0.2227],
        [0.0658, 0.5835, 0.1271, 0.1848],
        [0.3953, 0.6325, 0.0875, 0.2414],
        [0.4975, 0.6273, 0.0966, 0.2312],
        [0.3308, 0.6190, 0.0471, 0.1910],
        [0.4872, 0.5283, 0.0337, 0.0272],
        [0.3136, 0.5320, 0.0829, 0.3240],
        [0.3875, 0.4462, 0.0236, 0.0839],
        [0.1881, 0.5017, 0.0230, 0.0375],
        [0.2137, 0.5364, 0.0317, 0.2542],
        [0.0438, 0.7717, 0.0224, 0.1073],
        [0.0318, 0.7781, 0.0201, 0.1090],
        [0.2894, 0.3100, 0.0218, 0.0514],
        [0.1134, 0.8316, 0.0573, 0.2105],
        [0.4431, 0.5167, 0.0178, 0.0529],
        [0.3483, 0.5288, 0.0150, 0.0294],
        [0.6120, 0.4784, 0.0222, 0.0414],
        [0.4662, 0.4879, 0.0152, 0.0393],
        [0.4000, 0.6471, 0.1962, 0.2088]]), 'labels': tensor([64, 72, 72, 62, 62, 62, 62,  1,  1, 78, 82, 84, 84, 85, 86, 86, 62, 86,
        86, 67]), 'image_id': tensor([139]), 'area': tenso

KeyError: 0

In [60]:
selected_train_data = [d for d in dataset if d[1]['image_id'] in list(train_data.keys())]
selected_val_data = [d for d in dataset if d[1]['image_id'] in list(val_data.keys())]

In [62]:
selected_val_data

[]

In [25]:
if args.distributed:
        sampler_train = DistributedSampler(dataset_train)
        sampler_val = DistributedSampler(dataset_val, shuffle=False)
else:
    sampler_train = torch.utils.data.RandomSampler(dataset_train)
    sampler_val = torch.utils.data.SequentialSampler(dataset_val)

batch_sampler_train = torch.utils.data.BatchSampler(
        sampler_train, args.batch_size, drop_last=True)

data_loader_train = DataLoader(dataset_train, batch_sampler=batch_sampler_train,
                                   collate_fn=utils.collate_fn, num_workers=args.num_workers)
data_loader_val = DataLoader(dataset_val, args.batch_size, sampler=sampler_val,
                                 drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)

AttributeError: 'Namespace' object has no attribute 'batch_size'

In [ ]:
data_loader